In [2]:
import os
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch import nn, optim
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

data_dir = 'Dog Emotion'
batch_size = 32
num_epochs = 10
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = full_dataset.classes
num_classes = len(class_names)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}: Loss = {running_loss:.4f}, Accuracy = {acc*100:.2f}%")

torch.save(model.state_dict(), 'dog_emotion.pth')
print("\n model saved as dog_emotion.pth")

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, preds = outputs.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

f1 = f1_score(all_labels, all_preds, average='weighted')
report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)

print(f"\nF1-score (weighted): {f1:.4f}")
print("\nReport:\n", report)


C:\Users\DKpro\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\DKpro\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/10: 100%|████████████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  3.01it/s]


Epoch 1: Loss = 74.2330, Accuracy = 69.09%


Epoch 2/10: 100%|████████████████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.56it/s]


Epoch 2: Loss = 29.2875, Accuracy = 89.59%


Epoch 3/10: 100%|████████████████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.56it/s]


...

Epoch 8: Loss = 2.8315, Accuracy = 99.25%


Epoch 9/10: 100%|████████████████████████████████████████████████████████████████████| 100/100 [00:28<00:00,  3.55it/s]


Epoch 9: Loss = 2.6998, Accuracy = 99.28%


Epoch 10/10: 100%|███████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.40it/s]


Epoch 10: Loss = 2.7094, Accuracy = 99.16%

 model saved as dog_emotion.pth

F1-score (weighted): 0.8479

Report:
               precision    recall  f1-score   support

       angry     0.8278    0.8371    0.8324       178
       happy     0.8739    0.8981    0.8858       216
     relaxed     0.8394    0.7642    0.8000       212
         sad     0.8488    0.8969    0.8722       194

    accuracy                         0.8488       800
   macro avg     0.8475    0.8491    0.8476       800
weighted avg     0.8484    0.8488    0.8479       800

